# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Откроем файл с данными и изучим общую информацию

In [4]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter

In [5]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод:**

В датафрейме присутствуют 3 типа даных, столбцы 'days_employed' и 'total_income' имею меньшее количество строк (есть пропуски).


## Предобработка данных

### Обработка пропусков

In [7]:
data.loc[data.days_employed >0, 'days_employed'] = data.loc[data.days_employed >0, 'days_employed'] / 24 #переводим часы стажа в дни, все значения в часах оказались положительными, поэтому применим условие >0 для перевода в дни

data.loc[data['children'] < 0, 'children'] = 0 #заменяем отрицательное количество детей на предполагаемый 0

data['days_employed'] = np.abs(data['days_employed']) #избавляемся от отрицательных значений в столбце 'days_employed'

children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median() #избавляемся от случайных значений в виде 20 детей

data['children'] = data['children'].replace(20, children_median) #меняем сомнительное значение 20 детей на медианное
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1.0,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0.0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3.0,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0.0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0.0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0.0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0.0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2.0,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0.0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [8]:
for dob_years in data['dob_years'].unique(): #заменяем пропущенные значения в столбце 'days_employed' на значение медианы, которое зависит от cтажа
    median = data.loc[data['dob_years'] == dob_years, 'days_employed'].median()
    data.loc[(data['days_employed'].isna()) & (data['dob_years'] == dob_years) , 'days_employed'] = median 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  float64
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


In [9]:
total_income_meadian = data['total_income'].median() #медиана по столбцу "общий доход"
data['total_income'] = data['total_income'].fillna(total_income_meadian) # присваиваем пропускам значение медианы по столбцу 'total_income'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  float64
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


**Вывод:**

Предоставленные для анализа данные нельзя назвать качественными, в 2 столбцах типа float64 были пропуски, в столбце "стаж" некоторые строки принимали значение в часах, вместо дней. Интрументами библиотек pandas и numpy мы восполнили и исправили недостающие и некорректные данные, теперь анализ данных будет объективнее и точнее.

### Замена типа данных

In [10]:
data['days_employed'] = data['days_employed'].astype(int) #методом astype() меняет тип данных в столбце со стажем c float64 на int64 
data['total_income'] = data['total_income'].astype(int) #методом astype() меняет тип данных в столбце с доходами c float64 на int64
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  float64
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**2 столбца типа float64 были приведены к целочисленным значениям методом astype()**

### Обработка дубликатов

In [11]:
#data.duplicated().sum()
data['education'] = data['education'].str.lower() #приводим все значения столбца к нижнему регистру
#data['education'].value_counts() #убеждаемся, что была произведена замена и объединение значений
#data.duplicated().sum()
data['family_status'] = data['family_status'].str.lower() #приводим все значения столбца к нижнему регистру
#data['family_status'].value_counts() # всего найдет 71 дубликат 
data = data.drop_duplicates()
data.duplicated().sum()


0

**Вывод:**

Всего был обнаружен 71 дубликат, цифра небольшая относительно всего датафрейма, тем не менее они были удалены методом drop_duplicates(), который отлично справился с этой задачей.


### Лемматизация

In [12]:
m = Mystem()
data['lem_purpose'] = data['purpose'].apply(m.lemmatize) #создание нового столбца
data['lem_purpose']

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: lem_purpose, Length: 21454, dtype: object

In [13]:
list_of_lemmas = []
for element in data['purpose']: #определим и отсортируем уникальные значения в столбце 'purpose'
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)
unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True) #сортируем в порядке убывания уникалные значения столбца 'purpose'

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

In [14]:
stemmed_purposes = ['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба'] #создаем список самых популярных слов для дальнейшей группировки
dict_stemmed_purposes = {v:k for k,v in enumerate(stemmed_purposes)}
dict_stemmed_purposes['недвижимость'] = 1
#print(dict_stemmed_purposes)

data['purpose_category'] = 0 #проставляем номер категории в столбце 'purpose_category' 1-4 по столбцу 'lem_purpose'
for row in range(len(data)):
    for purpose in dict_stemmed_purposes:
        if purpose in data.iloc[row, 12]:
            data.iloc[row, 13] = dict_stemmed_purposes[purpose]
#data.head()
data[data['purpose_category'] == 0].count().sum() #номера категорий 1-4 присвоены всем строкам датафрейма
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,purpose_category
0,1.0,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",1
1,1.0,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",2
2,0.0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",1
3,3.0,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",3
4,0.0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",4
5,0.0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",1
6,0.0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",1
7,0.0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",3
8,2.0,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",4
9,0.0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",1


**Вывод:**

Самыми часто встречающимися словами стали: 'недвижимость'/'жилье','автомобиль','образование','свадьба', отсюда мы делаем вывод, что люди чаще всего берут кредиты на жилье, авто, учебу, автомобили и образование.

### Категоризация данных

In [15]:
education_dict = data[['education','education_id']].drop_duplicates().reset_index(drop=True) #education_dictionary
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [16]:
family_status_dict = data[['family_status','family_status_id']].drop_duplicates().reset_index(drop=True) #family_status_dictionary
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [17]:
income_type_dict = data[['income_type']].drop_duplicates().reset_index(drop=True) #income_type_dictionary
income_type_dict

,income_type
0,сотрудник
1,пенсионер
2,компаньон
3,госслужащий
4,безработный
5,предприниматель
6,студент
7,в декрете


In [18]:
gender_dict = data[['gender']].drop_duplicates().reset_index(drop=True) #gender_dictionary
gender_dict

,gender
0,F
1,M
2,XNA


In [19]:
purpose_dict = data[['purpose','purpose_category']].drop_duplicates().reset_index(drop=True) #purpose_dict
purpose_dict

,purpose,purpose_category
0,покупка жилья,1
1,приобретение автомобиля,2
2,дополнительное образование,3
3,сыграть свадьбу,4
4,операции с жильем,1
5,образование,3
6,на проведение свадьбы,4
7,покупка жилья для семьи,1
8,покупка недвижимости,1
9,покупка коммерческой недвижимости,1


**Вывод:**

Категоризировали датафрейм по следующим данным:  
-'education' и 'family_status_id' (изначально уже была категоризация по ним);  
-по виду занятости ('income_type');  
-по половой приндлежности ('gender');  
-по цели получения кредита ('purpose_category').

## Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
data.pivot_table(index=['children'], values='debt', aggfunc=['count','mean'])

,count,mean
,debt,debt
children,,
0.0,14214,0.075419
1.0,4808,0.092346
2.0,2052,0.094542
3.0,330,0.081818
4.0,41,0.097561
5.0,9,0.000000


**Вывод:**

Заемщиков без детей значительно больше и они реже задерживают оплату кредитов, чем люди с детьми, это логично, в семьях с детьми расходы на содержание семьи значительно выше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
fam_st = data.groupby('family_status')['debt'].agg(['count','mean']) #сгруппируем таблицу по семейному положению, в показателях выведем столбец 'debt'
fam_st.sort_values('mean', ascending=False) #отсортируем в порядке убыванию по проценту событий, когда хоть раз задерживалась выплата по кредиту

,count,mean
family_status,,
не женат / не замужем,2810,0.097509
гражданский брак,4151,0.093471
женат / замужем,12339,0.075452
в разводе,1195,0.071130
вдовец / вдова,959,0.065693


**Вывод:**

Не женатые люди чаще всех имели задолженность по кредитам, очень близко к ним люди в гражданском браке, реже всех  - вдовцы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
data['total_income_group'] = pd.qcut(data['total_income'], q = [0, 0.25, 0.5, 0.75, 1], labels = ['очень низкий','низкий','средний','высокий'])
data.pivot_table(index=['total_income_group'], values='debt', aggfunc=['count','mean'])


,count,mean
,debt,debt
total_income_group,,
очень низкий,5364,0.079605
низкий,6415,0.085269
средний,4311,0.089074
высокий,5364,0.071402


**Вывод:**

Люди с высоким доходом реже всех задерживают платежи по кредиту, люди с самым низким доходом платят лучше (стабильнее), чем люди с низким/средним доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
(data.groupby('purpose_category')['debt'].agg(['count','mean'])).sort_values('mean', ascending=False)



,count,mean
purpose_category,,
2,4306,0.093590
3,4013,0.092200
4,2324,0.080034
1,10811,0.072334


**Вывод:**

{'недвижимость/жилье': 1, 'автомобиль': 2, 'образование': 3, 'свадьба': 4}  
**Самые ответственные - те, которые брали кредит на жилье, самые безответственные 2 и 3 группа (на образование и на авто)**

## Общий вывод

Отвечая на вопросы проектной работы я сознательно использовал 2 метода для построения сводных таблиц (groupby и pivot_table).  В целом по проекту можно сделать вывод, что самые ответственные заемщики - это люди без детей, с самым высоким и с самым низким доходом, вдовцы, либо в разводе, и те, которые берут кредит на жилье. Самые безответственные  - не женатые люди, либо находящиеся в гражданском браке с низким или средним доходом и берущие кредиты на образование, либо авто.